In [ ]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
# import asyncio

In [ ]:
start_date = "2000-01-01"
end_date = "2010-12-31"
# comids = [2043493, 20873280, 7590453]
comids = [2043493, 20873280, 7590453]


nwm = NWM(start_date=start_date, end_date=end_date, comids=comids)
scheduler = LocalCluster(n_workers=10, threads_per_worker=2, processes=False)
scheduler

In [ ]:
%%time
# %%prun
nwm.request_timeseries(scheduler=scheduler, optimize=True, nwm_21=True)
nwm.data
# 10y, 1comid: 14min CPU, 19min
# 10y, 2comid: 21min CPU, 30min
# 10y, 3comid: 32min CPU, 46min

In [ ]:
%%time
df = nwm.set_output(return_dataframe=False)
nwm.output

In [ ]:
%%time
nwm.request_timeseries(scheduler=scheduler, optimize=False, nwm_2=True)
nwm.data

In [ ]:
%%time
nwm.set_output()
nwm.output


-----------------------------------

In [6]:
import xarray as xr
import fsspec
import dask
import s3fs
from datetime import datetime
from rechunker import rechunk

from dask.distributed import Client, LocalCluster

In [7]:
n_workers = 10
scheduler = LocalCluster(n_workers=n_workers, threads_per_worker=2, processes=False)
client = Client(scheduler)
client.dashboard_link

C:\Users\dsmith\Anaconda3\envs\hms_kube\lib\site-packages\distributed\node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50113 instead
  warnings.warn(


'http://10.104.45.242:50113/status'

In [8]:
# nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr'
start_date = datetime.fromisoformat("2000-01-01")
end_date = datetime.fromisoformat("2010-12-31")

# STREAMFLOW 
# comids = [2043493, 20873280, 7590453]
# comids = [2043493]
# variables = ["streamflow", "velocity"]

# LAKE
comids = [166757656]
variables = ["inflow", "outflow", "water_sfc_elev"]

start_string = f"{start_date.year}-{start_date.month}-{start_date.day}"
end_string = f"{end_date.year}-{end_date.month}-{end_date.day}"

In [9]:
%%time
# drop_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral"]
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True)
ds

distributed.worker - WARNING - Compute Failed
Function:  subgraph_callable-5640f0e7-ab33-4f5f-900e-8392942d
args:      (ImplicitToExplicitIndexingAdapter(array=CopyOnWriteArray(array=LazilyIndexedArray(array=_ElementwiseFunctionArray(_ElementwiseFunctionArray(LazilyIndexedArray(array=<xarray.backends.zarr.ZarrArrayWrapper object at 0x000001CFCABA9AC0>, key=BasicIndexer((slice(None, None, None), slice(None, None, None)))), func=functools.partial(<function _apply_mask at 0x000001CFC724FAF0>, encoded_fill_values={-999900}, decoded_fill_value=nan, dtype=dtype('float64')), dtype=dtype('float64')), func=functools.partial(<function _scale_offset_decoding at 0x000001CFC724FB80>, scale_factor=0.009999999776482582, add_offset=0.0, dtype=<class 'numpy.float64'>), dtype=dtype('float64')), key=BasicIndexer((slice(None, None, None), slice(None, None, None)))))), (slice(249984, 258048, None), slice(5000, 5500, None)))
kwargs:    {}
Exception: 'FSTimeoutError()'

distributed.worker - WARNING - Compute

CPU times: total: 5.62 s
Wall time: 13.4 s


<xarray.Dataset>
Dimensions:         (feature_id: 5783, time: 367439)
Coordinates:
  * feature_id      (feature_id) int32 491 531 747 ... 947070204 1021092845
    latitude        (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    crs             |S1 ...
    inflow          (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    outflow         (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    water_sfc_elev  (time, feature_id) float32 dask.array<chunksize=(8064, 500), meta=np.ndarray>
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

In [10]:
%%time
# Option one, default with comid selection and time slice happening at the same time with dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

Start: 2000-1-1, End: 2010-12-31
CPU times: total: 49.9 s
Wall time: 1min 26s


<xarray.Dataset>
Dimensions:         (time: 96432, feature_id: 1)
Coordinates:
  * feature_id      (feature_id) int32 166757656
    latitude        (feature_id) float32 26.95
    longitude       (feature_id) float32 -80.83
  * time            (time) datetime64[ns] 2000-01-01 ... 2010-12-31T23:00:00
Data variables:
    inflow          (time, feature_id) float64 50.45 50.42 50.4 ... 26.1 26.09
    outflow         (time, feature_id) float64 82.41 82.41 82.4 ... 59.52 59.51
    water_sfc_elev  (time, feature_id) float32 9.965 9.965 9.965 ... 9.646 9.646
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

In [ ]:
%%time
ds = ds.sel(time=slice(start_string, end_string))
ds_features_0 = (ds.where(ds.feature_id.isin(comids), drop=True))
ds_features = ds_features_0.copy()
ds_features = ds_features.drop('crs')

# Option two, rechunk the data
dim_chunk_sizes = {'feature_id': len(comids), 'time': len(ds_features.time)}
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
chunk_plan = {}
for vv in ds_features.variables:
    if vv in ['streamflow', 'velocity']:
        chunk_plan[vv] = tuple((dim_chunk_sizes[tt] for tt in ds_features[vv].dims))
    else: 
        chunk_plan[vv] = ds_features[vv].shape
    ds_features[vv].encoding['chunks'] = None

max_mem = f"{format(0.9 * 56 / n_workers, '.2f')}GB"
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
intermediate = "intermediate.zarr"
target = "target.zarr"

rechunked = rechunk(ds_features, target_chunks=chunk_plan, target_store=target,
                    max_mem=max_mem,
                    temp_store=intermediate)
result = rechunked.execute(retries=10)
ds = xr.open_zarr(target)
ds_data = ds[variables].load(optimize_graph=True, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
# Option three, no dask slicing
ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
data = data.sel(time=slice(start_string, end_string))
data

In [ ]:
%%time
# zarr_file = nwm_url
# s3 = s3fs.S3FileSystem(anon=True)
# store = s3fs.S3Map(root=zarr_file, s3=s3, check=False)
# ds_nwm_chrtout = xr.open_zarr(store=store, consolidated=True)
# ds_nwm_chrtout

In [ ]:
%%time
ds_nwm_feature = ds_nwm_chrtout.isel(feature_id==f'{self.comids}'.encode())
streamflow_nwm = ds_nwm_feature.streamflow.load()
streamflow_nwm_df = streamflow_nwm.squeeze('feature_id').to_dataframe()
streamflow_nwm_df

In [ ]:
%%time
variables = ["streamflow", "velocity"]
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
# with dask.config.set(**{'array.slicing.split_large_chunks': True}):
ds_data = ds[variables].sel(feature_id=nwm.comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
ds_data = ds.sel(feature_id=nwm.comids)
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
ds_data

In [ ]:
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
print(f"Start: {start_string}, End: {end_string}")
ds_data

In [ ]:
all_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
d_v = list(set(all_variables) - set(variables))
d_v